In [5]:
import camelot
import pandas as pd


In [8]:
# Check if dependencies are installed correctly
import tkinter
import ghostscript

import ctypes
from ctypes.util import find_library
import os 
os.environ['PATH'] += os.pathsep + r'C:\Program Files\gs\gs10.03.1\bin'
find_library("".join(("gsdll", str(ctypes.sizeof(ctypes.c_voidp) * 8), ".dll")))

'C:\\Program Files\\gs\\gs10.03.1\\bin\\gsdll64.dll'

## Getting started

### First tests

In [32]:
tables = camelot.read_pdf('Example.pdf')
print(tables)
print(tables[0])
print(tables[0].parsing_report)
tables[0].df

<TableList n=1>
<Table shape=(2, 4)>
{'accuracy': 88.64, 'whitespace': 0.0, 'order': 1, 'page': 1}


,0,1,2,3
0,Datum,Erläuterung,Betrag Soll EUR,Betrag Haben EUR
1,02.04.2024 Überweisung online\n02.04.2024 Über...,"Kontostand am 28.03.2024, Auszug Nr. 3\nKat...","-500,00\n -25,60\n -...","17.467,17"


In [38]:
#TODO: Extract table row lines correctly

In [37]:
import camelot

# Path to the PDF file
file_path = "Example.pdf"

# Read the PDF file using the 'stream' flavor
tables = camelot.read_pdf(file_path, flavor='stream', pages='all')

tables[0].df
# # Print the tables
# for i, table in enumerate(tables):
#     print(f"Table {i}")
#     print(table.df)


,0,1,2
0,Kontoauszug 4/2024,,Seite 1 von 8
1,"S-Giro-Online 8017535, DE38 7655 0000 0008 017...",,
2,"Freistellungsauftrag über 125 EUR,",,
3,bereits beanspruchter Betrag 0 EUR,,
4,Datum\nErläuterung,Betrag Soll EUR,Betrag Haben EUR
5,"Kontostand am 28.03.2024, Auszug Nr. 3",,"17.467,17"
6,02.04.2024 Überweisung online,"-500,00",
7,Katrin Wagner Miete März 2024 DATUM 29.03.2024...,,
8,UHR,,
9,02.04.2024 Überweisung online,"-25,60",


### Trying to adjust for missing table lines in bank statement pdf

In [17]:
def process_tables_from_pdf(pdf_path):
    # Extract tables from the PDF
    tables = camelot.read_pdf(pdf_path, pages='all')
    all_data = []

    for table in tables:
        df = table.df  # Convert to DataFrame
        all_data.append(df)

    # Combine all tables into a single DataFrame
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Perform any additional processing with pandas
    # For example, rename columns, filter rows, etc.
    combined_df.columns = combined_df.iloc[0]
    combined_df = combined_df.drop(0).reset_index(drop=True)
    
    return combined_df

# Path to your PDF file
pdf_path = 'Example.pdf'

# Process tables and get a combined DataFrame
processed_data = process_tables_from_pdf(pdf_path)

# Print or save the processed data
print(type(processed_data))

<class 'pandas.core.frame.DataFrame'>


In [23]:
processed_data.columns

Index(['Datum', 'Erläuterung', 'Betrag Soll EUR', 'Betrag Haben EUR'], dtype='object', name=0)

In [26]:
len(processed_data.loc[0, :]["Datum"])

249

In [27]:
def clean_ocr_output(text):
    lines = text.split('\n')
    cleaned_lines = [line.strip() for line in lines if line.strip() != '']
    return cleaned_lines

In [29]:
date_entries = clean_ocr_output(processed_data.loc[0, :]["Datum"])
print(len(date_entries))

In [30]:

for column in processed_data.columns: 
    print(f"COLUMN: {column}")
    print(f'Number of entries: {len(clean_ocr_output(processed_data.loc[0, :]["Datum"]))}')
    print(processed_data.loc[0, :][column])
    

COLUMN: Datum
Number of entries: 9
02.04.2024 Überweisung online
02.04.2024 Überweisung online
02.04.2024 Überweisung online
02.04.2024 Lastschrift
02.04.2024 Lastschrift
02.04.2024 Kartenzahlung Euro
03.04.2024 Kartenzahlung Euro
03.04.2024 Kartenzahlung Euro
04.04.2024 Dauerauftrag
COLUMN: Erläuterung
Number of entries: 9
Kontostand am 28.03.2024, Auszug Nr.    3
Katrin Wagner Miete März 2024 DATUM 29.03.2024, 10.04 
UHR
Klarna Bank AB (publ) XA4TIOMP7XHDEWEBHPSEFNNVXB, 
Liefera ndo.de, ISEB04DE0E7, W20UC6 DATUM 
01.04.2024, 19.47 UHR
Katrin Wagner Essen gehen DATUM 01.04.2024, 19.49 
UHR
Rheinische Friedrich-Wilhelms-Unive Universitaet Bonn 
Angebot 910001-1777, Hochschulsport ID631-134, 
Entgelt, Wintersemester 2023/2024 Telefon 0228-734185 
ID631-134 BS-612413 Gläubiger-ID: 
DE79ZZZ00000105150
AMAZON MEDIA EU S.A R.L. D01-1511433-8613434 Kindle 
Svcs 4PP 8UZJ6SRSSH23U 4PP8UZJ6SRSSH23U 
6JVUZBr,(?62)yHSEpH?yzobxha8W( Gläubiger-ID: 
DE78ZZZ00000561650
Backerei Kamps//Bonn/DE 2024-03